# Switch

## This notebook mainly focus on following. Turn the part on by setting 1 otherwise it will load the pre-train result.

In [1]:
switch = {
    'DataInput and Sampling':1,
    'Single feature Search':0,
    'Single feature Evaluate':1,
    'Two feature Search':0,
    'Two feature Evaluate':1,
    'Three feature Search':0,
    'Three feature Evaluate':1,
    'Four feature Search':0,
    'Four feature Evaluate':1,
    'RF feature Search':0,
    'RF feature Evaluate':1,
}

In [ ]:
import numpy as np
from program import pretreat
import pandas as pd
from tqdm import tqdm
from itertools import combinations
from copy import deepcopy as dcp

In [ ]:
if (switch['DataInput and Sampling']):
    df,title = pretreat.input()
    X=df[:,1:19]
    print(X.shape)
    y = np.array(df[:,0],dtype=np.int) 
    pretreat.kfoldsampling(X,y)

# DataInput and Sampling

In [4]:
set_name = ['','s','u','o']#No sampling,smote,under sampling,over sampling
train=[]
test=[]
for i in set_name:
    train.append((
                np.load(r'result/train_set/'+i+'xtrain.npy'),
                np.load(r'result/train_set/'+i+'ytrain.npy')
    ))
    test.append((
                np.load(r'result/test_set/'+i+'xtest.npy'),
                np.load(r'result/test_set/'+i+'ytest.npy')
    ))

In [5]:
for i in range(len(train)):
    print('Nums of sample:',train[i][0].shape[0],'Feature of each sample:',train[i][0].shape[1])

Nums of sample: 44436 Feature of each sample: 18
Nums of sample: 87680 Feature of each sample: 18
Nums of sample: 1192 Feature of each sample: 18
Nums of sample: 87680 Feature of each sample: 18


# Single feature

In [6]:
from program import classify
from program import metrics

In [7]:
if (switch['Single feature Search']):
    for j in range(len(set_name)):
        performance_single = []
        for k in classify.clf:
            for i in tqdm(range(train[0][0].shape[1])):
                performance_single.append(metrics.performance(k,train[j][0],train[j][1],test[j][0],test[j][1],[i]))
        np.save(r'result/performance/'+set_name[j]+'single_performance.npy',performance_single)

In [8]:
if (switch['Single feature Evaluate']):
    pd.set_option('precision', 2)
    single_index = []
    for j in range(len(set_name)):
        index = []
        per = np.load(r'result/performance/'+set_name[j]+'single_performance.npy',allow_pickle = True)
        show = []
        for i in range(4):
            no = int(per.shape[0]/4)
            temp = per[no*i:no*(i+1),:]
            if (np.max(temp[:,2])<0.95):
                index.append(np.argmax(temp[:,2]))
            else:
                for k in temp:
                    if (k[2]<0.95):
                        k[3] = 0
                index.append(np.argmax(temp[:,3]))
            show.append(index[-1]+i*no)
        print(set_name[j]+'sampling index:',[k+1 for k in index])
        print(pd.DataFrame(per[show,2:],index=('Decision Tree','Adaboost','Xgboost','Gboost'),
              columns=('Recall','Precision','F1','FN','FP','Train Time(s)','Test Time(s)')))
        print('---------------------------------------------------------------')
        single_index.append(dcp(show))
    np.save(r'result/index/single_index.npy',single_index)
    #The index shown here starts at 1, but the saved index starts at 0
else:
    single_index = np.load(r'result/index/single_index.npy')

sampling index: [10, 10, 10, 10]
              Recall Precision    F1   FN    FP Train Time(s) Test Time(s)
Decision Tree    0.8      0.94  0.86  121    28           0.1        0.003
Adaboost        0.78      0.97  0.87  130    15             4          1.2
Xgboost         0.78      0.94  0.86  129    29           2.6         0.19
Gboost          0.79      0.13  0.22  128  3131           3.6         0.21
---------------------------------------------------------------
ssampling index: [1, 5, 5, 5]
              Recall Precision    F1  FN    FP Train Time(s) Test Time(s)
Decision Tree   0.98      0.32  0.48  12  1263          0.19        0.003
Adaboost        0.98      0.32  0.48  12  1234           7.7          1.1
Xgboost         0.98      0.32  0.48  12  1234           4.4         0.16
Gboost          0.98      0.32  0.48  12  1243           7.2         0.13
---------------------------------------------------------------
usampling index: [1, 1, 5, 1]
              Recall Precision    

# Two feature

In [9]:
if (switch['Two feature Search']):
    for j in range(len(set_name)):
        performance_two = []
        for k in classify.clf:
            for i in tqdm(combinations(range(0,train[0][0].shape[1]), 2)):
                performance_two.append(metrics.performance(k,train[j][0],train[j][1],test[j][0],test[j][1],i))
        np.save(r'result/performance/'+set_name[j]+'two_performance.npy',performance_two)

In [10]:
if (switch['Two feature Evaluate']):
    pd.set_option('precision', 2)
    two_index = []
    all = [i for i in combinations(range(0,train[0][0].shape[1]), 2)]
    for j in range(len(set_name)):
        index = []
        per = np.load(r'result/performance/'+set_name[j]+'two_performance.npy',allow_pickle = True)
        show = []
        for i in range(4):
            no = int(per.shape[0]/4)
            temp = per[no*i:no*(i+1),:]
            if (np.max(temp[:,2])<0.95):
                index.append(all[np.argmax(temp[:,2])])
                show.append(np.argmax(temp[:,2])+i*no)
            else:
                for k in temp:
                    if (k[2]<0.95):
                        k[3] = 0
                index.append(all[np.argmax(temp[:,3])])
                show.append(np.argmax(temp[:,3])+i*no)
        print(set_name[j]+'sampling index:',index)#The index here start from 0
        print(pd.DataFrame(per[show,2:],index=('Decision Tree','Adaboost','Xgboost','Gboost'),
              columns=('Recall','Precision','F1','FN','FP','Train Time(s)','Test Time(s)')))
        print('---------------------------------------------------------------')
        two_index.append(dcp(show))
    np.save(r'result/index/two_index.npy',two_index)
else:
    two_index = np.load(r'result/index/two_index.npy')

sampling index: [(2, 9), (4, 9), (1, 9), (1, 9)]
              Recall Precision    F1  FN  FP Train Time(s) Test Time(s)
Decision Tree   0.91      0.93  0.92  56  42         0.096        0.003
Adaboost         0.9      0.93  0.91  62  43           5.4          1.1
Xgboost         0.91      0.96  0.93  51  25           2.6         0.24
Gboost          0.91      0.93  0.92  55  43           5.3         0.16
---------------------------------------------------------------
ssampling index: [(0, 9), (0, 9), (0, 9), (0, 9)]
              Recall Precision    F1  FN   FP Train Time(s) Test Time(s)
Decision Tree   0.98      0.68   0.8  11  281          0.19        0.003
Adaboost        0.99      0.67   0.8   7  293            11          1.1
Xgboost         0.98      0.68   0.8  12  280           4.9         0.17
Gboost          0.98      0.69  0.81  14  267            11         0.13
---------------------------------------------------------------
usampling index: [(0, 8), (0, 9), (0, 9), (0, 9)

# Three feature

In [11]:
from sklearn.feature_selection import RFECV as RFE

In [12]:
if (switch['Three feature Search']):
    three_index = []
    index_list = np.array([i for i in range(train[0][0].shape[1])])
    for j in range(len(set_name)):
        performance_three = []
        index = []
        for k in tqdm(classify.clf):
            rfe = RFE(estimator = k, min_features_to_select = 3, step = 1, cv = 3, scoring = 'recall')
            temp = metrics.performance_rfe(rfe,train[j][0],train[j][1],test[j][0],test[j][1])
            performance_three.append(temp[1:])
            index.append(index_list[temp[0]])
        three_index.append(dcp(index))
        np.save(r'result/performance/'+set_name[j]+'three_performance.npy',performance_three)
        np.save(r'result/index/three_index.npy',three_index)

In [13]:
if (switch['Three feature Evaluate']):
    pd.set_option('precision', 2)
    three_index = np.load(r'result/index/three_index.npy',allow_pickle = True)
    for j in range(len(set_name)):
        performance_three = np.load(r'result/performance/'+set_name[j]+'three_performance.npy',allow_pickle = True)
        print('Feature index:')
        for k in three_index[j]:
            print(k)
        print(pd.DataFrame(performance_three[:,2:],index=('Decision Tree','Adaboost','Xgboost','Gboost'),
              columns=('Recall','Precision','F1','FN','FP','Train Time(s)','Test Time(s)')))
        print('---------------------------------------------------------------')

Feature index:
[2 4 9]
[ 0  1  2  3  4  7  8  9 10 11 14 15 16 17]
[ 0  1  2  3  4  5  7  8  9 10 11 13]
[2 7 9]
              Recall Precision    F1  FN  FP Train Time(s) Test Time(s)
Decision Tree   0.95      0.97  0.96  31  15            13        0.004
Adaboost        0.97      0.99  0.98  19   7       7.5e+02          2.1
Xgboost         0.97      0.98  0.97  19  11         4e+02         0.19
Gboost          0.96      0.98  0.97  21  14       4.3e+02         0.13
---------------------------------------------------------------
Feature index:
[ 1  3  4  8  9 10 16]
[ 0  1  2  4  7  8  9 10 11]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
[ 0  1  2  3  4  7  9 10 11]
              Recall Precision    F1  FN  FP Train Time(s) Test Time(s)
Decision Tree   0.97      0.89  0.93  18  74            42        0.007
Adaboost        0.99      0.86  0.92   8  98       1.7e+03          1.6
Xgboost         0.98      0.95  0.96  13  33       7.5e+02         0.23
Gboost          0.98      

# Four feature

In [14]:
if (switch['Four feature Search']):
    four_index = []
    index_list = np.array([i for i in range(train[0][0].shape[1])])
    for j in range(len(set_name)):
        performance_four = []
        index = []
        for k in tqdm(classify.clf):
            rfe = RFE(estimator = k, min_features_to_select = 4, step = 1, cv = 3, scoring = 'recall')
            temp = metrics.performance_rfe(rfe,train[j][0],train[j][1],test[j][0],test[j][1])
            performance_four.append(temp[1:])
            index.append(index_list[temp[0]])
        four_index.append(dcp(index))
        np.save(r'result/performance/'+set_name[j]+'four_performance.npy',performance_four)
        np.save(r'result/index/four_index.npy',four_index)

In [15]:
if (switch['Four feature Evaluate']):
    pd.set_option('precision', 2)
    four_index = np.load(r'result/index/four_index.npy',allow_pickle = True)
    for j in range(len(set_name)):
        performance_four = np.load(r'result/performance/'+set_name[j]+'four_performance.npy',allow_pickle = True)
        print('Feature index:')
        for k in four_index[j]:
            print(k)
        print(pd.DataFrame(performance_four[:,2:],index=('Decision Tree','Adaboost','Xgboost','Gboost'),
              columns=('Recall','Precision','F1','FN','FP','Train Time(s)','Test Time(s)')))
        print('---------------------------------------------------------------')

Feature index:
[1 2 4 9]
[ 0  1  2  3  4  7  8  9 10 11 14 15 16 17]
[ 0  1  2  3  4  5  7  8  9 10 11 13]
[ 1  2  4  9 10]
              Recall Precision    F1  FN  FP Train Time(s) Test Time(s)
Decision Tree   0.95      0.98  0.96  31  14            13        0.006
Adaboost        0.97      0.99  0.98  19   7       7.4e+02          1.9
Xgboost         0.97      0.98  0.97  19  11         4e+02         0.17
Gboost          0.97      0.98  0.97  20  14         4e+02        0.099
---------------------------------------------------------------
Feature index:
[ 1  3  4  8  9 10 16]
[ 0  1  2  4  7  8  9 10 11]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
[ 0  1  2  3  4  7  8  9 10 11 16]
              Recall Precision    F1  FN  FP Train Time(s) Test Time(s)
Decision Tree   0.97      0.89  0.93  18  70            40        0.007
Adaboost        0.99      0.86  0.92   8  98       1.7e+03          1.6
Xgboost         0.98      0.95  0.96  13  33       7.5e+02         0.24
Gboost   

# Random Forest

In [16]:
if (switch['RF feature Search']):
    rf_performance = []
    for j in tqdm(range(len(set_name))):
        rf_performance.append(metrics.performance_rfc(classify.rfc,train[j][0],train[j][1],test[j][0],test[j][1]))
    np.save(r'result/performance/rfc_performance.npy',rf_performance)
    temp = [i[2] for i in rf_performance]
    index = []
    for j in temp:
        index.append(np.argsort(j))
    print(index)
    np.save(r'result/index/rfc_index.npy',index)
    rf_performance = []
    for j in tqdm(range(len(set_name))):
        rf_performance.append(metrics.performance_rfc(classify.rfc,train[j][0][:,index[j][-4:]],train[j][1],test[j][0][:,index[j][-4:]],test[j][1]))
    np.save(r'result/performance/rfc_4_performance.npy',rf_performance)

In [17]:
if (switch['RF feature Evaluate']):
    pd.set_option('precision', 2)
    per = np.load(r'result/performance/rfc_performance.npy',allow_pickle = True)
    show = []
    for j in range(len(set_name)):
        show.append(per[j][3:])
    print('All feature')
    print(pd.DataFrame(show,index=('No sampling','Smote','Under sampling','Over Sampling'),
          columns=('Recall','Precision','F1','FN','FP','Train Time(s)','Test Time(s)')))
    print('---------------------------------------------------------------')
    print('Feature index:')
    index = np.load(r'result/index/rfc_index.npy')
    print(index[:,-4:])
    per = np.load(r'result/performance/rfc_4_performance.npy',allow_pickle = True)
    show = []
    for j in range(len(set_name)):
        show.append(per[j][3:])
    print(pd.DataFrame(show,index=('No sampling','Smote','Under sampling','Over Sampling'),
          columns=('Recall','Precision','F1','FN','FP','Train Time(s)','Test Time(s)')))

All feature
                Recall  Precision    F1  FN   FP  Train Time(s)  Test Time(s)
No sampling       0.95       0.99  0.97  27    5         111.47          3.24
Smote             0.98       0.94  0.96  12   38         305.76          3.12
Under sampling    0.99       0.66  0.79   5  309           2.49          2.88
Over Sampling     0.97       0.96  0.97  16   26         123.11          2.96
---------------------------------------------------------------
Feature index:
[[ 1 14  8  9]
 [ 9  7  0  4]
 [ 9  7  0  4]
 [ 9  7  4  0]]
                Recall  Precision    F1  FN   FP  Train Time(s)  Test Time(s)
No sampling       0.94       0.98  0.96  36   14          89.08          4.02
Smote             0.98       0.76  0.85  13  189         152.62          2.92
Under sampling    0.98       0.59  0.73  11  411           1.84          2.67
Over Sampling     0.96       0.80  0.87  22  143          71.00          3.05
